In [1]:
import matplotlib
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import math
import scipy as sp
from scipy import interpolate
from MyUnit import SIUnit, NaturalUnit
from scipy.interpolate import interp1d

# choose the specific CLASS version
classy_path = '/a/home/cc/students/physics/chensun/tomerv_storage/Code/class_axion/python/build/lib.linux-x86_64-2.7'
import sys
sys.path.insert(1, classy_path)
from classy import Class

import classy
classy.__file__


'/a/home/cc/students/physics/chensun/tomerv_storage/Code/class_axion/python/build/lib.linux-x86_64-2.7/classy.so'

In [2]:
def joinstring(list):
    res = ""
    for x in list[:-1]:
        res = res+str(x)+","
    res = res + str(list[-1])
    return res

# test = [1.,3.,7.,10.]
# joinstring(test)
# '1.0,3.0,7.0,10.0'

In [3]:
# np.sqrt(6.)
# # 2.449489742783178

In [4]:
# draw the potential
_eV_Mpc_=1.5637496259e29;
_sqrt_8pi_=5.013256549;
f=4e-4 # in Mpl,r units
m=1e-20 # in eV units
phi_ini = (np.sqrt(6)-1e-3)*f
#phi_ini = (np.sqrt(6)-1e-6)*f

def V(phi):
    return 1./2.*m**2*phi**2 - 1./24.*m**2/f**2*phi**4
phi_list = np.linspace(0.,3.)
plt.subplots()
plt.plot(phi_list, V(phi_list*f))

V_list = np.linspace(0,1.5)
plt.plot([np.sqrt(6)]*50,V_list*3e-47, '--')
# hill top corresponds to 
# phi = sqrt(6) f

<IPython.core.display.Javascript object>

In [ ]:
# Define your cosmology (what is not specified will be set to CLASS default parameters)
params = {
    #'gauge':'synchronous',
    'gauge':'newtonian',
    'input_verbose' : 1,
    'background_verbose' : 2,
    'thermodynamics_verbose': 1,
    'perturbations_verbose': 2,
    'h': 0.673,
    'omega_b': 0.02205,
    #'omega_cdm': 0.1199,
    'omega_cdm': 0.01,
    'tau_reio': 0.089,
    'A_s':math.exp(3.089)*10**-10,
    'n_s':0.9603,
    'N_ur' : 2.03351,
    'N_ncdm' : 1,
    'm_ncdm' : 0.06,
    'T_ncdm' : 0.715985, 
    'T_cmb' : 2.2755,
    ## Output setup
    #'lensing': 'yes',
    #'l_max_scalars': 3000, 
    #'output': 'tCl,pCl,lCl,mPk',
    #'output': 'mTk, vTk',
    #'compute damping scale': 'Y'
    #'k_output_values':'1.,7.,70.'
}

#k_list = [0.01,10.,1000.]
k_crit = 75.35
k_list = [k_crit/20.,k_crit, k_crit*2.]

settings_scf = {
    'phi_ini_scf': phi_ini,
    'scf_m': m,
    'scf_f': f,
    'scf_w': 0,
    'scf_phiprime_ini': 1e-50,
    'scf_tau_end_over_tau_crit': 8.,
    'output': 'mTk, vTk',
    'k_output_values':'%s' %(joinstring(k_list)),
    'P_k_max_1/Mpc': 7.2e-6 # need to set this otherwise most of the time is spend on computing k ~ 0-1
}

try:
    cosmo.struct_cleanup()
except:
    pass
# Create an instance of the CLASS wrapper
cosmo = Class()

# Set the parameters to the cosmological code
cosmo.set(params)
cosmo.set(settings_scf)
cosmo.compute()

print cosmo.age()

In [ ]:
np.log(1./0.95)/np.log(10)

In [ ]:
# # debugging
# import matplotlib
# %matplotlib notebook
# import numpy as np
# import matplotlib.pyplot as plt


# plt.subplots()
# plt.yscale('log')
# plt.xscale('log')
# path = '/a/home/cc/students/physics/chensun/tomerv_storage/Code/class_axion/test_phi_of_tau_after.txt'
# data = []
# with open(path, 'r') as f:
#     for line in f:
#         line_list = line.split()
#         data.append(line_list)
# data = np.asarray(data)
# data = data.astype(float)
# plt.plot(data[:,0], np.abs(data[:,1]), '.',markersize=.1)

# # plt.subplots()
# # plt.yscale('log')
# # plt.xscale('log')
# # path = '/a/home/cc/students/physics/chensun/tomerv_storage/Code/class_axion/test_V_of_tau_after.txt'
# # data = []
# # with open(path, 'r') as f:
# #     for line in f:
# #         line_list = line.split()
# #         data.append(line_list)
# # data = np.asarray(data)
# # data = data.astype(float)
# # plt.plot(data[:,0], np.abs(data[:,1]), '.',markersize=.1)
# # end of debugging

In [ ]:
background = cosmo.get_background()
print '----background keys: '
print background.keys()
print '\n\n'

thermodynamics = cosmo.get_thermodynamics()
print '----thermodynamics keys: '
print thermodynamics.keys()
print '\n\n'

perturbations = cosmo.get_perturbations()
print '----perturbations keys: '
print perturbations['scalar'][0].viewkeys()
print '\n\n'

# print '----CMB power spectra keys: '
# print cosmo.lensed_cl(3000).viewkeys()
# print '\n\n'

# z_rec = cosmo.get_current_derived_parameters(['z_rec'])['z_rec']
# print z_rec

z_eq = cosmo.get_current_derived_parameters(['z_eq'])['z_eq']
print("z_eq=%g" %z_eq)

# tau_eq = np.interp(z_eq, background['z'], background['conf. time [Mpc]'])
tau_eq = interp1d(background['z'], background['conf. time [Mpc]'], kind='cubic')(z_eq)
print("tau_eq=%g" %tau_eq)

In [ ]:
# estimate of k_crit

m_Mpc = m*_eV_Mpc_
tau_crit = np.sqrt(2./m_Mpc*tau_eq*(1.+z_eq))
a_crit = np.sqrt(2./(1.+z_eq)/m_Mpc/tau_eq)
z_crit = 1./a_crit - 1.
k_crit = m_Mpc * a_crit
print('k_crit = %g'%(k_crit))
print('a_crit = %g'%(a_crit))

## Background

In [ ]:
plt.subplots()
plt.xscale('log')
plt.yscale('log')
plt.xlabel('a')
a_list = 1/(1+background['z'])
mask1=a_list > a_crit
mask2=a_list < 10.*a_crit
mask=mask1*mask2
plt.plot(a_list[mask], background['(.)rho_scf'][mask], '-', label=r"$\rho_{\phi}$")
plt.legend(loc='best')
#plt.xlim(5e-8, 1e-5)
#plt.ylim(1e4, 1e11)

In [ ]:
plt.subplots()
plt.xscale('log')
plt.yscale('log')
plt.xlabel('a')
plt.plot(1/(1+background['z'][mask]), background['(.)rho_scf'][mask]/background['(.)rho_tot'][mask], '-', label=r"$\Omega_{\phi}$")
plt.legend(loc='best')

In [ ]:
plt.subplots()
plt.xscale('log')
plt.xlabel('a')
plt.plot(1/(1+background['z'][mask]), background['(.)p_scf'][mask]/background['(.)rho_scf'][mask], '-', label=r"$w_{\phi}$")
plt.legend(loc='best')
#plt.xlim(5e-8, 1e-4)
#plt.ylim(1e4, 1e11)

In [ ]:
plt.subplots()
#plt.xscale('log')
#plt.yscale('log')
plt.xlabel('a')
a_list = 1/(1+background['z'])
mask1=a_list > a_crit/10.
mask2=a_list < 7.*a_crit
mask=mask1*mask2
plt.plot(a_list[mask], background['phi_scf'][mask], '-', label=r"$\phi$")
# plt.plot(1/(1+background['z'][mask]), np.abs(background['phi_scf'][mask]), '-', label=r"$\phi$")
plt.legend(loc='best')
#plt.xlim(5e-8, 1e-4)
#plt.ylim(1e4, 1e11)

plt.subplots()
#plt.xscale('log')
plt.xlabel('a')
plt.plot(1/(1+background['z'][mask]), background['phi\'_scf'][mask], '-', label=r"$\phi'$")
plt.legend(loc='best')
#plt.xlim(5e-8, 1e-4)
#plt.ylim(1e4, 1e11)

In [ ]:
plt.subplots()
#plt.xscale('log')
#plt.yscale('log')
plt.xlabel('a')

a_list = 1/(1+background['z'])
mask1=a_list > a_crit/10.
mask2=a_list < 7.*a_crit
mask=mask1*mask2
plt.plot(a_list[mask], background['V_scf'][mask], '-', label=r"$V(\phi)$")
plt.legend(loc='best')

In [ ]:
plt.subplots()
#plt.xscale('log')
#plt.yscale('log')
plt.xlabel('\phi')

a_list = 1/(1+background['z'])
mask1=a_list > a_crit/10.
mask2=a_list < 7.*a_crit
mask=mask1*mask2
#plt.plot(a_list[mask], background['V_scf'][mask], '-', label=r"$V(\phi)$")
#plt.plot(a_list[mask], background['phi_scf'][mask], '-', label=r"$V(\phi)$")
plt.plot(background['phi_scf'][mask],background['V_scf'][mask], '-', label=r"$V(\phi)$")
plt.legend(loc='best')

## Perturbation

In [ ]:
a_list = perturbations['scalar'][0]['a']
mask1=a_list > a_crit
mask2=a_list < 7.*a_crit
mask_pert=mask1*mask2

plt.subplots()
plt.xscale('log')
plt.yscale('log')
plt.plot(perturbations['scalar'][0]['a'][mask_pert], np.abs(perturbations['scalar'][0]['delta_scf'][mask_pert]))

In [ ]:
plt.subplots()
plt.xscale('log')
plt.yscale('log')
plt.xlabel('a')

a_list = 1/(1+background['z'])
mask1=a_list > a_crit/10.
mask2=a_list < 10.*a_crit
mask=mask1*mask2
plt.plot(1/(1+background['z'][mask]), background['(.)rho_scf'][mask], '-', label=r"$\rho_{\phi}$")

for i in range(len(perturbations['scalar'])):
    a_list = perturbations['scalar'][i]['a']
    mask1=a_list > 0.1*a_crit
    mask2=a_list < 7.5*a_crit
    mask_pert=mask2*mask1
    plt.plot(perturbations['scalar'][i]['a'][mask_pert], 
             np.abs(perturbations['scalar'][i]['delta_scf'][mask_pert]/perturbations['scalar'][i]['delta_scf'][0]), 
             label=r"$\delta_\phi$,k=%g/Mpc"%k_list[i])
# i=2
# plt.plot(perturbations['scalar'][i]['a'], 
#          np.abs(perturbations['scalar'][i]['delta_scf']), 
#          label=r"$\delta_\phi$,k=%g/Mpc"%k_list[i])

plt.legend(loc='best')
#plt.xlim(1e-8, 1e-5)
#plt.ylim(1e-1, 1e13)